# 🧭 Overview

This tutorial introduces the five main classes that structure your atmospheric retrieval setup:

### - Instrument -- defines the observing setup (resolution, wavelength range, data files).
### - ModelConfig -- configures the underlying physical and chemical model settings.
### - IOConfig -- controls input/output and file management for retrieval runs.
### - Retrieval_params -- defines which parameters will be retrieved (e.g., gases, P–T structure, clouds).
### - ArgsGen -- Combines all configuration instances and the data into one object.


| Class                  | Purpose                                                                                                             | Example Contents                                                     |
| ---------------------- | ------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------- |
| **`Instrument`**       | Defines the observational setup (e.g., spectral resolution, wavelength range...).             | `fwhm`, `Rfile`                         |
| **`ModelConfig`**      | Controls the physical and chemical modeling configuration, including line lists, distance, and key modeling flags.  | `xpath`, `xlist`, `dist`, `malk`, `do_fudge`                         |
| **`IOConfig`**         | Manages input/output settings, filenames, checkpoints, and runtime logs.                                            | `runname`, `outdir`, `finalout`, `chaindump`                         |
| **`Retrieval_params`** | Defines the full set of retrievable parameters, including gases, P–T structure, and clouds across multiple patches. | `gaslist`, `ptype`, `cloud_name`, `cloud_type`, `cloudpatch_index`   |
| **`ArgsGen`**          | Combines all configuration objects and observational data into a unified argument set for the retrieval run.        | `obspec`, linked configs (`re_params`, `model_config`, `instrument`) |


In [1]:
import os 
import utils
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings

## 🔭 1. Instrument Configuration
The Instrument class describes instrument mode of your data from a user defined R file — specify the data wavelength range, wavelength-dependent resolving power, tolerance parameter and scale factor flags.

In [2]:
# Define basic instrument parameters
wavelength_range = [0.95, 2.85]       # Wavelength coverage in microns
Rfile = './examples/example_data/G570D_R_file.txt'  # File with resolving power data
obspec = np.asfortranarray(np.loadtxt("examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true'))

# Create an Instrument instance
instrument_instance = utils.Instrument(wavelength_range=wavelength_range, R_file=Rfile,obspec=obspec)
print(instrument_instance)


Instrument: 
------------
- wavelength_range : [0.95, 2.85]
- R_file : ./examples/example_data/G570D_R_file.txt
- R_data : {'R': array([ 81.05428479,  81.26206564,  81.36553273,  81.36508951,
        81.36624204,  81.36817893,  81.47468354,  81.58065025,
        81.58490566,  81.69228356,  81.79990613,  81.90777154,
        81.91386816,  82.02310794,  82.13180693,  82.14010481,
        82.35144704,  82.46084152,  82.47016524,  82.68184597,
        82.79192073,  82.90298054,  83.01350121,  83.125     ,
        83.33660822,  83.44767793,  83.65963855,  83.77028245,
        83.88189448,  84.09356287,  84.2047491 ,  84.41676611,
        84.62827771,  84.73914982,  84.9510095 ,  85.16236655,
        85.27292899,  85.48463357,  85.69583825,  85.90728184,
        86.11822733,  86.32941176,  86.54009988,  86.64932591,
        86.96219226,  87.27432591,  87.38230994,  87.5927278 ,
        87.9052278 ,  88.11478996,  88.32459207,  88.63709207,
        88.84604191,  89.1589057 ,  89.47104191,  89

## 🪐 2. Model Configuration
The ModelConfig class defines global physical and numerical settings used by the forward model — e.g., where to find opacity files, whether to include alkali broadening, or how to treat cloud properties.

In [3]:
# Define basic model parameters
samplemode='mcmc'
do_fudge=1
cloudpath='../cloud_pickles/'


# Create a ModelConfig instance
model_config_instance = utils.ModelConfig(
    samplemode, do_fudge, cloudpath=cloudpath
)

# Additional settings
model_config_instance.dist = 5.84
model_config_instance.xlist = 'data/gaslistRox.dat'
model_config_instance.xpath = '../../Linelists/'
model_config_instance.do_bff = 0
# model_config_instance.malk = 1
# model_config_instance.ch4 = 0
model_config_instance.update_dictionary()

print(model_config_instance)


ModelConfig: 
------------
- use_disort : 0
- do_fudge : 1
- malk : 0
- mch4 : 0
- do_bff : 0
- fresh : 0
- xpath : ../../Linelists/
- xlist : data/gaslistRox.dat
- dist : 5.84
- pfile : data/LSR1835_eqpt.dat
- cloudpath : /Users/fw23aao/Downloads/cloud_pickles/
- do_scales : True
- do_shift : True

(MCMC): 
------------
- ndim : None
- nwalkers : None
- nburn : 10000
- niter : 30000



## 💾 3. I/O Configuration
The IOConfig class handles filenames and output management.
It ensures every run produces consistent file outputs, checkpoints, and logs.

In [4]:
# Create a IO_config instance
io_config_instance = utils.IOConfig()

# Additional settings
io_config_instance.outdir="../fei/test/"
io_config_instance.runname='new_test'
io_config_instance.update_dictionary()

print(io_config_instance)


IOConfig: 
------------
- runname : new_test
- outdir : ../fei/test/
- finalout : new_test.pk1
- chaindump : new_test_last_nc.pic
- picdump : new_test_snapshot.pic
- statfile : status_ball_new_test.txt
- rfile : runtimes_new_test.dat
- runtest : 1
- make_arg_pickle : 2



## ☁️ 4. Retrieval Parameter Configuration
The Retrieval_params class defines all the parameters that will be explored by the retrieval sampler — from gases and P–T structure to clouds and spatial patches.

In [5]:
# Chemistry setup
chemeq = 0  # 0 = free chemistry, 1 = chemical equilibrium
gaslist = ['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'na', 'k'] # List of gases included in the retrieval

# Gas type list:
#   'U' = uniform mixing ratio
#   'N' = parameterized vertically varying profile
# This allows different gases to have different vertical distributions

gastype_list = ['U', 'U', 'U', 'U', 'U', 'N', 'U', 'U']

# PT structure
ptype = 77

# Clouds
do_clouds = 1 # Enable or disable cloud modeling, 1 = include clouds, 0 = clear atmosphere
npatches = 2 # Number of atmospheric patches 

# List of all cloud species or parameterized cloud models.
# You can mix empirical clouds (e.g. 'powerlaw', 'grey') and physical Mie scattering clouds.
cloud_name = ['powerlaw', 'grey', 'MgSiO3.mieff']

# Cloud type specifies the geometrical setup:
cloud_type = ['slab', 'deck', 'slab']



# Specify which patch each cloud appears in.
# Each entry in the list corresponds to the same index in `cloud_name`.
# Example:
#   [1]    → only in patch 1
#   [2]    → only in patch 2
#   [1, 2] → appears in both patches

cloudpatch_index = [
    [1],      # 'powerlaw' cloud only in patch 1
    [1, 2],   # 'grey' cloud in both patches
    [1, 2]    # 'MgSiO3.mieff' cloud in both patches
]

# Particle size distribution for each cloud. Use 'None' for parameterized or grey clouds.
# For Mie scattering clouds, specify:'hansen' or 'log_normal' distributions.
particle_dis = ['None', 'None', 'hansen']



#===Clear+Cloudy PATCHY CLOUD SETUP========

"""
# Example of a simpler two-patch setup:
#   - Patch 1: clear
#   - Patch 2: Mie scattering MgSiO3 cloud

npatches = 2
cloud_name = ['clear', 'MgSiO3.mieff']
cloud_type = ['None', 'slab']

cloudpatch_index = [
    [1],  # Clear patch 1
    [2]   # Cloudy patch 2
]

particle_dis = [
    'None',   # Clear atmosphere
    'hansen'  # Mie scattering distribution
]
"""
# ============================

# Create retrieval parameter object, the deafault vrad and vsini are false.
re_params = utils.Retrieval_params(samplemode=samplemode, chemeq=chemeq, gaslist=gaslist,
    gastype_list=gastype_list,do_fudge=do_fudge, ptype=ptype, do_clouds=do_clouds,
    npatches=npatches, cloud_name=cloud_name, cloud_type=cloud_type,
    cloudpatch_index=cloudpatch_index, particle_dis=particle_dis,
    instrument=instrument_instance,vrad=True,vsini=True,fwhm=None)

print(re_params)


retrieval_param: 
------------
- gas : ['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'na', 'k']
- refinement_params : ['logg', 'r2d2', 'vrad', 'vsini', 'tolerance_parameter_1']
- pt : ['gamma', 'Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3']
- cloud : ['fcld', 'patch 1', 'patch 2']
  -- patch 1: ['tau_pcs', 'logp_pcs', 'dp_pcs', 'omega_pcs', 'alpha_pcs', 'logp_gcd', 'dp_gcd', 'omega_gcd', 'tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']
  -- patch 2: ['logp_gcd', 'dp_gcd', 'omega_gcd', 'tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']
- added_params: []



### 4.1 check all retrieval parameters

In [6]:
# re_params.dictionary
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary) 
print(all_params)

['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'p_ref_feh', 'alpha_feh', 'na', 'k', 'logg', 'r2d2', 'vrad', 'vsini', 'tolerance_parameter_1', 'gamma', 'Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3', 'fcld', 'tau_pcs', 'logp_pcs', 'dp_pcs', 'omega_pcs', 'alpha_pcs', 'logp_gcd', 'dp_gcd', 'omega_gcd', 'tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']


### 4.2 initialize the parameters
You can modify or customize the initialization of all retrieval parameters like this:

In [7]:
re_params.dictionary['gas']['h2o']['params']['log_abund']['distribution']=['normal',-3.5,0.5]
re_params.dictionary['gas']['co']['params']['log_abund']['distribution']=['normal',-8.0,0.5]
re_params.dictionary['refinement_params']['params']['logg']['distribution']=['normal',4.9,0.1]

In [8]:
re_params.dictionary

{'gas': {'h2o': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -3.5, 0.5],
     'range': [-12, 0],
     'prior': None}}},
  'co': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -8.0, 0.5],
     'range': [-12, 0],
     'prior': None}}},
  'tio': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': [-12, 0],
     'prior': None}}},
  'vo': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': [-12, 0],
     'prior': None}}},
  'crh': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': [-12, 0],
     'prior': None}}},
  'feh': {'gastype': 'N',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': [-12, 0],
    

In [9]:
re_params.dictionary['pt']['params']['Tint']

{'initialization': None,
 'distribution': ['normal', 1200, 200],
 'range': [0, 5000],
 'prior': None}

## ⚙️ 5. Generating the Retrieval Arguments
Once all configuration objects are set up, the next step is to bundle them together into a unified args_instance.
This object holds everything the retrieval engine needs — model parameters, instrument details, observed data, and model options.
This is handled by the ArgsGen class.

In [10]:
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [11]:
args_instance.cloud_opaname

array(['powerlaw', 'grey', 'MgSiO3.mieff'], dtype=object)

In [12]:
args_instance.cloudmap

array([[1, 1, 1],
       [0, 1, 1]])

## ⚙️ 6. Prior settings

In [13]:
args_instance.R_priorange=[1,10.0]
args_instance.Mass_priorange=[10.0,30.0]

re_params.dictionary['gas']['h2o']['params']['log_abund']['range']=[-6,0]
re_params.dictionary['gas']['co']['params']['log_abund']['range']=[-10,-4]

## 🚀 7. Start the retrieval ! (on cluster...)
Once all configuration objects and arguments have been initialized, you’re ready to run the retrieval.
This is done through the brewster_reterieval_run function in the retrieval_run module.
This function takes care of:
Building the forward model
Initializing the sampler (e.g., MCMC)
Running the retrieval loop
Saving outputs and diagnostics

## 🔍 8. diagnostic functions on priors and log likelihood

### 8.1 # Check which parameters fail priors check 

In [14]:
import Priors
import test_module

In [15]:
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary)

if re_params.samplemode=='mcmc':
    
    model_config_instance.ndim=len(all_params)
    model_config_instance.nwalkers=len(all_params)*16

    r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
    re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal',r2d2,0.1*r2d2]


    def tolerance_parameter_customized_distribution(x):
        return np.log10((np.random.rand(x)* (max(args_instance.obspec[2,:]**2)*(0.1 - 0.01))) + (0.01*min(args_instance.obspec[2,10::3]**2))) 

    for i in range(len(all_params)):
        if all_params[i].startswith('tolerance_parameter'):
            re_params.dictionary['refinement_params']['params'][all_params[i]]['distribution']=['customized',tolerance_parameter_customized_distribution]

    if model_config_instance.fresh == 0:
        p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance,args_instance)

In [16]:
params_master = namedtuple('params',all_params)
params_instance = params_master(*p0[0,:])
params_instance

params(h2o=-2.156438573568557, co=-7.7967253841841195, tio=-3.6727586257520057, vo=-3.9001098082882173, crh=-5.000269941414939, feh=-3.9706755048772675, p_ref_feh=-1.1159777340470403, alpha_feh=0.3207920541814486, na=-3.5719248854796417, k=-4.6780490806033965, logg=4.972744313901431, r2d2=1.4207421563092585e-19, vrad=-1.5992228719172388, vsini=59.37748065657148, tolerance_parameter_1=-33.427509313905226, gamma=50.53890494226708, Tint=1420.2257694521431, alpha=1.7798766881363761, lndelta=-0.6568332211622187, T1=981.0893943598776, T2=812.2357905932005, T3=1179.2529329791246, fcld=0.95464916037362, tau_pcs=9.445640525479009, logp_pcs=0.14043034925619585, dp_pcs=0.1195730080016465, omega_pcs=0.6692942917184588, alpha_pcs=-1.2337766141836863, logp_gcd=0.9042756932947653, dp_gcd=0.04180045815223096, omega_gcd=0.3543633975860182, tau_mcs_MgSiO3=11.515439455859845, logp_mcs_MgSiO3=-0.492380160374834, dp_mcs_MgSiO3=0.06594872237868639, hansen_a_mcs_MgSiO3=-1.3311401718592455, hansen_b_mcs_MgSiO

In [17]:
Priors_instance =Priors.Priors(params_instance,re_params,args_instance)

In [18]:
print(Priors_instance.statement, Priors_instance.priors)

False -inf


In [19]:
print(Priors_instance)

All priors considered in the retrieval:
------------
- Parameter Priors: Defined by user ranges
  * h2o: [-6, 0]
  * co: [-10, -4]
  * tio: [-12, 0]
  * vo: [-12, 0]
  * crh: [-12, 0]
  * feh: [-12, 0]
  * p_ref_feh: [-4, 2.4]
  * alpha_feh: [0, 1]
  * na: [-12, 0]
  * k: [-12, 0]
  * logg: [0, 6]
  * r2d2: [0, 1]
  * vrad: [-250, 250]
  * vsini: [0, 100]
  * tolerance_parameter_1: No prior range defined
  * gamma: [0, 5000]
  * Tint: [0, 5000]
  * alpha: [1, 2]
  * lndelta: [-20, 0]
  * T1: [0, 5000]
  * T2: [0, 5000]
  * T3: [0, 5000]
  * fcld: [0, 1]
  * tau_pcs: [0, 100]
  * logp_pcs: [-4, 2.4]
  * dp_pcs: No prior range defined
  * omega_pcs: [0, 1]
  * alpha_pcs: [-10, 10]
  * logp_gcd: [-4, 2.4]
  * dp_gcd: [0, 7]
  * omega_gcd: [0, 1]
  * tau_mcs_MgSiO3: [0, 100]
  * logp_mcs_MgSiO3: [-4, 2.4]
  * dp_mcs_MgSiO3: No prior range defined
  * hansen_a_mcs_MgSiO3: [-3, 3]
  * hansen_b_mcs_MgSiO3: [0, 1]
--------------------
- Post-processing Priors:
  * T-profile check: (min(T) > 1.

### 8.2 Check the lnlike of input statevector

In [20]:
settings.cia = args_instance.cia
settings.linelist= utils.get_opacities(args_instance.gaslist,args_instance.w1,args_instance.w2,args_instance.press,args_instance.xpath,args_instance.xlist,args_instance.malk)
settings.cloudata = args_instance.cloudata

In [21]:
lnlike = test_module.lnlike(params_instance,re_params)
print(lnlike)

-3449075.1334233955
